# imports

In [2]:
import numpy as np
from conv import Conv3x3
from maxpool import MaxPool2
from softmax import Softmax

from matplotlib import pyplot as plt

# load mnist handwritten dataset from tensorflow
import tensorflow as tf
mnist = tf.keras.datasets.mnist

In [3]:
# split the data into train and test
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images[:1000]
train_labels = train_labels[:1000]
test_images = test_images[:1000]
test_labels = test_labels[:1000]

In [4]:
# Normalize the images.
train_images = train_images / 255.0
test_images = test_images / 255.0

In [5]:
# making a model 

conv = Conv3x3(num_filters=9)                   # 9 filters, of size 3x3 and stride 1 without padding
pool = MaxPool2()                  # Pooling layer with size 2x2 and stride 2
softmax = Softmax(input_len=13 * 13 * 9, nodes=10)  # Softmax layer with 10 classes

In [6]:
# forward pass
def forward(image, label):
  '''
  Completes a forward pass of the CNN and calculates the accuracy and
  cross-entropy loss.
  - image is a 2d numpy array
  - label is a digit
  '''
  # Transform the grayscale image from [0, 255] to [-0.5, 0.5] to make it easier
  # to work with.
  out = conv.forward(image )
  out = pool.forward( out )
  out = softmax.forward( out )

  # Compute cross-entropy loss and accuracy.
  loss =  -np.log(out[label])
  acc =  1 if np.argmax(out) == label else 0

  return out, loss, acc

In [7]:
def train(im, label, lr=.001):
  '''
  A training step on the given image and label.
  Shall return the cross-entropy loss and accuracy.
  - image is a 2d numpy array
  - label is a digit
  - lr is the learning rate
  '''
  # Forward
  out, loss, acc = forward(im, label)

  # Calculate initial gradient
  gradient = np.zeros(10)
  gradient[label] = -1 / out[label]

  # Backprop
  gradient = softmax.backprop(gradient, lr)
  gradient = pool.backprop(gradient)
  gradient = conv.backprop(gradient, lr)

  return loss, acc

In [8]:
# training the model
for epoch in range(3):
  print('--- Epoch %d ---' % (epoch + 1))

  # Shuffle the training data
  permutation = np.random.permutation(len(train_images))
  train_images = train_images[permutation]
  train_labels = train_labels[permutation]

  # Train!
  loss = 0
  num_correct = 0

  for i, (im, label) in enumerate(zip(train_images, train_labels)):
    if i % 100 == 0:
      print(
        '[Step %d] Past 100 steps: Average Loss %.3f | Accuracy: %d%%' %
        (i, loss / 100, num_correct)
      )
      loss = 0
      num_correct = 0

    l, acc = train(im, label)
    loss += l
    num_correct  += acc

--- Epoch 1 ---
[Step 0] Past 100 steps: Average Loss 0.000 | Accuracy: 0%
[Step 100] Past 100 steps: Average Loss 2.295 | Accuracy: 19%
[Step 200] Past 100 steps: Average Loss 2.250 | Accuracy: 47%
[Step 300] Past 100 steps: Average Loss 2.234 | Accuracy: 40%
[Step 400] Past 100 steps: Average Loss 2.195 | Accuracy: 50%
[Step 500] Past 100 steps: Average Loss 2.151 | Accuracy: 52%
[Step 600] Past 100 steps: Average Loss 2.091 | Accuracy: 51%
[Step 700] Past 100 steps: Average Loss 2.036 | Accuracy: 49%
[Step 800] Past 100 steps: Average Loss 2.014 | Accuracy: 66%
[Step 900] Past 100 steps: Average Loss 1.928 | Accuracy: 70%
--- Epoch 2 ---
[Step 0] Past 100 steps: Average Loss 0.000 | Accuracy: 0%
[Step 100] Past 100 steps: Average Loss 1.720 | Accuracy: 67%
[Step 200] Past 100 steps: Average Loss 1.619 | Accuracy: 72%
[Step 300] Past 100 steps: Average Loss 1.487 | Accuracy: 73%
[Step 400] Past 100 steps: Average Loss 1.276 | Accuracy: 84%
[Step 500] Past 100 steps: Average Loss 1.30

### Training Set Accuracy

In [9]:
loss = 0
num_correct = 0
for im, label in zip(train_images, train_labels):
  _, l, acc = forward(im, label)
  loss += l
  num_correct += acc

num_trains = len(train_images)
print('Train Accuracy:', num_correct / num_trains)

Train Accuracy: 0.862


### Test Set Accuracy

In [10]:
loss = 0
num_correct = 0
for im, label in zip(test_images, test_labels):
  _, l, acc = forward(im, label)
  loss += l
  num_correct += acc

num_tests = len(test_images)
print('Test Accuracy:', num_correct / num_tests)

Test Accuracy: 0.781


# I have also written a Full Neural Netwrok Module from Scratch.

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from  nn.nn import NeuralNetwork as network
from  nn import layers as layers
from  nn import losses as losses
from nn import activations as activations
from nn.metrics import Metrics as metrics
from nn import optimizers as optimizers

In [12]:
# Changing the dimesions of the input 
x_train = train_images.reshape(-1,1,28,28)
x_test = test_images.reshape(-1,1,28,28)

# one hot encoding the y labels
y_train = pd.get_dummies(train_labels).values
y_test = pd.get_dummies(test_labels).values

In [13]:
digits = network()

digits.add(layers.ConvLayer(input_shape=(28,28,1), filters=9, kernel_size=(3,3), padding='valid', use_bias=True, strides=1))  
digits.add(layers.MaxPool2D(pool_size=(2,2), strides=(2,2)))


digits.add(layers.FlattenLayer())
digits.add(layers.DenseLayer( units=10, activation="softmax"))                

digits.compile(loss=losses.CategoricalCrossentropy(), optimizer=optimizers.Adam(lr=0.01), initializer="glorot_uniform", metrics=["accuracy", "precision"])

digits.summary()
digits.fit(x_train, y_train, epochs=10, batch_size=32, verbose=2)

Summary of the Neural Network
___________________________________________________________________________________________________________________
Layer (type)        Neurons #      Input Shape    Output Shape   Weights shape  Bias shape        Param #
Input               3              -              (3, None)      -              -                       0

ConvLayer           9              (28, 28, 1)    ((26, 26, 9),) (3, 3, 1, 9)   (9, 1)                 90

MaxPool2D           -              (26, 26, 9)    ((13, 13, 9),) -              -                       0

FlattenLayer        -              -              -              -              -                       0

DenseLayer          10             (1521,)        (10,)          (10, 1521)     (10, 1)             15220

Total params                                                                                        15310
Epoch 1-10 ======================> cost: 0.9918 accuracy: 0.7140 precision: 0.7293 
Epoch 2-10 ============

### Train Set Accuracy

In [14]:
pred_train_preb = digits.predict(x_train)

# get the index of the max value in each column
pred_train = np.argmax(pred_train_preb, axis=1)
y_train_ = np.argmax(y_train, axis=1)
print("Train Accuracy: ", metrics.accuracy(y_train_, pred_train))

Train Accuracy:  0.922


### Test Set Accuracy

In [15]:
pred_test_preb = digits.predict(x_test)

# get the index of the max value in each column
pred_test = np.argmax(pred_test_preb, axis=1)
y_test_ = np.argmax(y_test, axis=1)
print("Test Accuracy: ", metrics.accuracy(y_test_, pred_test))

Test Accuracy:  0.753
